# Setup

In [ ]:
!pip3 install wheel
!pip3 install --upgrade --force-reinstall client-sdk/python/

In [ ]:
import moonshine
from moonshine.api import default_api
from moonshine.model.argument import Argument
from moonshine.model.architecture_string import ArchitectureString as Architecture
from moonshine.model.artifact_type_string import ArtifactTypeString as ArtifactType
from moonshine.model.implant import Implant
from moonshine.model.implants_page import ImplantsPage
from moonshine.model.listener import Listener
from moonshine.model.listeners_page import ListenersPage
from moonshine.model.operating_system_string import OperatingSystemString as OperatingSystem
from moonshine.model.status import Status
from moonshine.model.task import Task
from moonshine.model.task_status_int32 import TaskStatusInt32 as TaskStatus
from moonshine.model.tasks_page import TasksPage
from moonshine.model.job import Job
from moonshine.model.job_status_int32 import JobStatusInt32 as JobStatus
from moonshine.model.job_success_int32 import JobSuccessInt32 as JobSuccess
from moonshine.model.service import Service
from moonshine.model.services_page import ServicesPage
from moonshine import ApiException

import os
import time
import json
import base64
from io import BytesIO
import zipfile
from pprint import pprint
import urllib3
urllib3.disable_warnings()

#### Create an instance of the API class

In [ ]:
configuration = moonshine.Configuration(
    host = "https://127.0.0.1:9000"
)
configuration.verify_ssl = False
api = default_api.DefaultApi(moonshine.ApiClient(configuration))

# Listener Management

#### List

In [ ]:
listeners = api.get_listeners(offset=0, limit=100)
pprint(listeners)

#### Create

Create a package (zip) containing the main listener script (init.lua), and any dependancies

In [ ]:
listener_package = BytesIO()
with zipfile.ZipFile(listener_package, mode="w",compression=zipfile.ZIP_DEFLATED) as zf:
    zf.writestr("init.lua", open("scripts/http/listener.lua", "rb").read())
    zf.writestr("httpd.lua", open("scripts/http/httpd.lua", "rb").read())

Do the same again for the corresponding implant script. Note, implants have a package size limit of 5KB.

In [ ]:
implant_package = BytesIO()
with zipfile.ZipFile(implant_package, mode="w",compression=zipfile.ZIP_DEFLATED) as zf:
    zf.writestr("init.lua", open("scripts/http/implant.lua", "rb").read())

Create a listener object that will be used to populate the create listener API call

In [ ]:
listener = Listener(
        name="HTTP Listener",
        package=base64.b64encode(listener_package.getvalue()).decode(),
        implant_package=base64.b64encode(implant_package.getvalue()).decode(),
        implant_connection_string="http://127.0.0.1:8080"
    )

listener.arguments = [
    Argument(base64.b64encode(b"0.0.0.0").decode()),
    Argument(base64.b64encode(b"8080").decode()),
]

Make API call

In [ ]:
listener = api.create_listener(listener)
pprint(listener.id)

Check if listener has been created

In [ ]:
listeners = api.get_listeners(offset=0, limit=100)
print(listeners.count)

Start the listener

In [ ]:
response = api.start_listener(listener.id)
pprint(response)

#### Stop listener

In [ ]:
response = api.stop_listener(listener.id)
pprint(response)

#### Delete listener

In [ ]:
listeners = api.get_listeners(offset=0, limit=100)
for listener in listeners.items:
    api.delete_listener(listener.id)

# Implant Management

#### Download implant

In [ ]:
from pathlib import Path
operating_system = OperatingSystem("windows")
architecture = Architecture("x86_64")
implant_type = ArtifactType("exe")
download_path = Path.home() / "Desktop" / ("implant." + operating_system.to_str() + "." + architecture.to_str() + "." + implant_type.to_str())

response = api.download_implant(listener.id, operating_system, architecture, implant_type)
open(download_path, 'wb').write(response.read())
os.chmod(download_path, 0o775)
print(f"{download_path} [{round(os.path.getsize(download_path) / 1024 / 1024, 2)} MB]")

#### Launch implant

In [ ]:
os.startfile(download_path)

#### List implants

In [ ]:
implants = api.get_implants(offset=0, limit=100)
pprint(implants.items[-1])

#### Submit task to an implant

First, create the lua script that will be executed on the implant, then create a Task object that will be used to submit the script to the server referencing the target implant.

In [ ]:
script='''

local implant = require 'implant'

print("Current implant checkin period is set to " .. implant.dwell() .. " seconds")

local checkin = 5

print("Setting implant checkin period to " .. checkin .. " seconds")

implant.dwell(checkin)

'''
task = Task(implants.items[-1].id, script=base64.b64encode(script.encode()).decode())

Now submit the Task to the server, which will then be collected by the implant on it's next check-in.

In [ ]:
task = api.create_task_for_implant(id=implants.items[-1].id, task=task)
pprint(task)

#### Check status and show result

Use a rudamentary loop to check every second for a response to the submitted task. Ideally the WebSocket based interface exposed by the server should be used to ingest pushed events from the server, which would allow asynchronous interaction.

In [ ]:
while True:
    response = api.get_task(id=task.id)
    if response.status == TaskStatus(3):
        print(base64.b64decode(response['output']).decode())
        break
    time.sleep(1)

## Script Examples

#### Loading a module

Lua modules can be loaded during runtime. First the module will need to be preloaded - which basically means that a copy is registered within the module sub-system, ready to be loaded into a script session via a call to 'require'.

In [ ]:
module = open("win32mod/win32mod.dll", "rb").read()
module_b64 = base64.b64encode(module).decode('ascii')

script='''

local mime = require "mime"

local mod = "win32mod"

print("Preloading lua module '" .. mod .. "' into runtime")

preload(1, mod, mime.unb64("%s"), false)

print("Preloaded modules:")
for index, name in pairs(preloaded()) do
  print("- " .. name)
end

print("\\nLoaded modules:")
for index, name in pairs(loaded()) do
  print("- " .. name)
end

''' % (module_b64)

task = Task(implants.items[-1].id, script=base64.b64encode(script.encode()).decode())

In [ ]:
task = api.create_task_for_implant(id=implants.items[-1].id, task=task)
pprint(task.id)

In [ ]:
while True:
    response = api.get_task(id=task.id)
    if response.status == TaskStatus(3):
        print(base64.b64decode(response['output']).decode())
        break
    time.sleep(1)

#### Calling a module function from a script

Once preloaded, the module is treated like any other lua module and can be used by scripts. The moon runtime includes some additinal functions that can be used to list both preloaded and loaded modules.

In [ ]:
script='''

local mod = "win32mod"

print("Loading lua module '" .. mod .. "'")

win32 = require(mod)

print("Loaded modules:")
for index, name in pairs(loaded()) do
  print("- " .. name)
end

print("Executing function exported from module")

win32.MessageBox("I'm a message", "I'm a title")

'''
task = Task(implants.items[-1].id, script=base64.b64encode(script.encode()).decode())

In [ ]:
task = api.create_task_for_implant(id=implants.items[-1].id, task=task)
pprint(task.id)

In [ ]:
while True:
    response = api.get_task(id=task.id)
    if response.status == TaskStatus(3):
        print(base64.b64decode(response['output']).decode())
        break
    time.sleep(1)

#### Foreign Function Interface (FFI)

LuaJIT, which is the default lua runtime used by Moon, comes with a build-in FFI library which allows external C functions and C data structures to be used from pure Lua code. This largely obviates the need to write tedious manual Lua/C bindings in C.

The following script performs the same action as the previously deployed module, but instead it is written using LuaJIT's FFI.

In [ ]:
script='''

local ffi = require("ffi")

ffi.cdef[[
int MessageBoxA(void *w, const char *txt, const char *cap, int type);
]]

ffi.C.MessageBoxA(nil, "I'm a message", "I'm a title", 0)

print("done")

'''
task = Task(implants.items[-1].id, script=base64.b64encode(script.encode()).decode())

In [ ]:
task = api.create_task_for_implant(id=implants.items[-1].id, task=task)
pprint(task.id)

In [ ]:
while True:
    response = api.get_task(id=task.id)
    if response.status == TaskStatus(3):
        print(base64.b64decode(response['output']).decode())
        break
    time.sleep(1)

#### Additional implant specific functions

In [ ]:
script='''

local implant = require "implant"

-- List jobs running on an implant

print("Jobs:")
for index, name in pairs(implant.jobs()) do
  print("- " .. name)
end

-- Change server connection string

local conn = implant.server()
print("Current server connection string is '" .. conn .. "'")
print("Setting server connection string to '" .. conn .. "'")
implant.server(conn)

-- Sleep for an arbitrary amount of seconds

print("Implant will sleep for 2 seconds")
implant.sleep(2)

-- Check if implant is in a "running" state i.e. not exiting

print("Implant is running ... " .. tostring(implant.running()))

'''
task = Task(implants.items[-1].id, script=base64.b64encode(script.encode()).decode())

In [ ]:
task = api.create_task_for_implant(id=implants.items[-1].id, task=task)
pprint(task.id)

In [ ]:
while True:
    response = api.get_task(id=task.id)
    if response.status == TaskStatus(3):
        print(base64.b64decode(response['output']).decode())
        break
    time.sleep(1)

#### Shutdown implant

To shutdown an implant, simply call the exit function. Note, currently the implant will shutdown without responding to the task.

In [ ]:
script='''

local implant = require "implant"

implant.exit()

'''
task = Task(implants.items[-1].id, script=base64.b64encode(script.encode()).decode())

In [ ]:
task = api.create_task_for_implant(id=implants.items[-1].id, task=task)
pprint(task.id)